In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
import glob
import os
import numpy as np
from PIL import Image
# from google.colab import drive
import torchvision.models as models
from tqdm import tqdm

In [2]:
# upload images from googledrive
# drive.mount('/content/gdrive')
# %cd ./gdrive/MyDrive/2021-DLCV-HW/HW1/
# !unzip hw1_data.zip

In [3]:
class HWIMGS(Dataset):
  def __init__(self, root, transform=None):
    " initial the dataset "
    self.image = None
    self.label = None
    self.filenames = []
    self.root = root;
    self.transform = transform

    #read filenames
    filenames = glob.glob(root+'/*.png')
    for fn in filenames:
        #store data, label
        reg = fn
        
#         colab:/*.png
#         jupyter:\*.png
        reg = fn.split('\\')[-1].split('_')[0]
#         print(reg)
        reg = int(reg)
        self.filenames.append((fn, reg))

    self.len = len(filenames)

  def __getitem__(self, index):
    " get a sample from the dataset "
    img_fn, label = self.filenames[index]
    # if torch.cuda.is_available():
    #   img_fn, label = img_fn.cuda(), label.cuda()
    image = Image.open(img_fn)
    if self.transform is not None:
      image = self.transform(image)
    
    return image, label
  
  def __len__(self):
    " Total number of sampler in the dataset "
    return self.len

In [4]:
from torchvision import transforms
trainset = HWIMGS(root='p1_data/train_50', transform=transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
#     transforms.RandomHorizontalFlip(p=0.5),
#     transforms.RandomRotation(2.8),
#     transforms.RandomPerspective(distortion_scale=0.5, p=0.5, interpolation=2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
]))

valset = HWIMGS(root='p1_data/val_50', transform=transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
]))
print(len(trainset))
print(len(valset))

22500
2500


In [5]:
trainset_loader = DataLoader(trainset, batch_size=32, shuffle=True)
valset_loader = DataLoader(valset, batch_size=32, shuffle=False)

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [7]:
# vgg16 = models.vgg16(pretrained=True)
# vgg16.classifier[6]= nn.Sequential(
#     nn.Dropout(0.2),
#     nn.Linear(4096, 50)
# )
# # vgg16.classifier[6]=nn.Linear(4096,50)
# vgg16 = vgg16.to(device)

In [8]:
from torchsummary import summary
resnet18 = models.resnet18(pretrained=True)
# print(resnet18)
# summary(resnet18, (3, 224, 224))
resnet18.fc= nn.Sequential(
    
    nn.Linear(512, 256),
    nn.ReLU(),
    nn.Dropout(0.3),
    
#     nn.Linear(256, 100),
#     nn.ReLU(),
#     nn.BatchNorm1d(100),
#     nn.Dropout(0.3),
    
    nn.Linear(256, 50)
)

for param in resnet18.parameters():
    resnet18.requires_grad = True
resnet18 = resnet18.to(device)
summary(resnet18, (3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]          36,864
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              ReLU-7           [-1, 64, 56, 56]               0
            Conv2d-8           [-1, 64, 56, 56]          36,864
       BatchNorm2d-9           [-1, 64, 56, 56]             128
             ReLU-10           [-1, 64, 56, 56]               0
       BasicBlock-11           [-1, 64, 56, 56]               0
           Conv2d-12           [-1, 64, 56, 56]          36,864
      BatchNorm2d-13           [-1, 64, 56, 56]             128
             ReLU-14           [-1, 64,

In [9]:
def train(model, epoch, log_interval=100):
    optimizer = optim.Adam(model.parameters(), lr=5e-4)
    scheduler = optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.9)
    criterion = nn.CrossEntropyLoss()
    model.train()  # Important: set training mode
    correct = 0
    
    iteration = 0
    for ep in range(epoch):
        correct = 0
        for batch_idx, (data, target) in enumerate(tqdm(trainset_loader), 1):
            
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
            pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()
            
#             if iteration % log_interval == 0:
        print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}, Accuracy: {}/{} ({:.0f}%)'.format(
            ep+1, batch_idx * len(data), len(trainset_loader.dataset),
            100. * batch_idx / len(trainset_loader), loss.item(),
            correct, len(trainset_loader.dataset),
            100. * correct / len(trainset_loader.dataset)))
#             iteration += 1
        val(model)
        val(model, scheduler) # Evaluate at the end of each epoch

In [10]:
def val(model, scheduler):
# def val(model):
    criterion = nn.CrossEntropyLoss()
    model.eval()  # Important: set evaluation mode
    val_loss = 0
    correct = 0
    with torch.no_grad(): # This will free the GPU memory used for back-prop
        for data, target in valset_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            val_loss += criterion(output, target).item() # sum up batch loss
            pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    val_loss /= len(valset_loader.dataset)
    print('\nVal set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        val_loss, correct, len(valset_loader.dataset),
        100. * correct / len(valset_loader.dataset)))
    scheduler.step()

In [ ]:
train(resnet18, epoch=30)

100%|████████████████████████████████████████████████████████████████████████████████| 704/704 [02:22<00:00,  4.94it/s]


Train Epoch: 1 [2816/22500 (100%)]	Loss: 2.079556, Accuracy: 10717/22500 (48%)


  0%|                                                                                  | 1/704 [00:00<02:17,  5.10it/s]


Val set: Average loss: 0.0465, Accuracy: 1436/2500 (57%)



100%|████████████████████████████████████████████████████████████████████████████████| 704/704 [02:22<00:00,  4.93it/s]


...

100%|████████████████████████████████████████████████████████████████████████████████| 704/704 [02:20<00:00,  5.00it/s]


Train Epoch: 3 [2816/22500 (100%)]	Loss: 1.157820, Accuracy: 10271/22500 (46%)


  0%|                                                                                  | 1/704 [00:00<02:21,  4.98it/s]


Val set: Average loss: 0.0503, Accuracy: 1379/2500 (55%)



 15%|███████████▉                                                                    | 105/704 [00:20<02:03,  4.85it/s]

In [ ]:
torch.save(resnet18, './resnet18_1.pth')